In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install py7zr

In [ ]:
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip',['.7z'], unpack_7zarchive)

In [ ]:
shutil.unpack_archive('../input/cifar-10/train.7z','/kaggle/temp/')
shutil.unpack_archive('../input/cifar-10/test.7z','/kaggle/temp/')

In [ ]:
train_labels = pd.read_csv('../input/cifar-10/trainLabels.csv',header='infer')
classes = train_labels['label'].unique()
print(classes)

In [ ]:
if os.path.exists('/kaggle/temp/validation/') == False:
    os.mkdir('/kaggle/temp/validation/')

parent_path_train='/kaggle/temp/train/'
parent_path_validation='/kaggle/temp/validation/'
parent_path_test='/kaggle/temp/test/'
for class1 in classes:
    path_train=os.path.join(parent_path_train,class1)
    if os.path.exists(path_train) == False:
        os.mkdir(path_train)
    path_validation=os.path.join(parent_path_validation,class1)
    if os.path.exists(path_validation) == False:
        os.mkdir(path_validation)

In [ ]:
for (int_ind,row) in train_labels.iterrows():
    
    id=str(row['id'])+'.png'
    source_path=os.path.join(parent_path_train,id)
    
    p=np.random.random()
    if p<= 0.8:
        target_path=os.path.join(parent_path_train,row['label'],id)
        os.replace(source_path,target_path)
    else:
        target_path=os.path.join(parent_path_validation,row['label'],id)
        os.replace(source_path,target_path)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input
import cv2

train = ImageDataGenerator(rescale=1./255, preprocessing_function = preprocess_input)
train_batch = train.flow_from_directory(parent_path_train, batch_size = 132, target_size=(72,72))

validation = ImageDataGenerator(rescale=1./255, preprocessing_function = preprocess_input)
val_batch = validation.flow_from_directory(parent_path_validation, batch_size = 132, target_size=(72,72))#, target_size = (299,299))

testgen = ImageDataGenerator(rescale = 1 / 255.0)
testflow = testgen.flow_from_directory('/kaggle/temp/', classes=['test'], batch_size = 132, target_size = (72, 72), shuffle = False)

In [ ]:
from keras import Sequential
from  keras.applications.xception import Xception
from keras.layers import Flatten, Dense
xception = Xception(include_top=False,
                weights = "imagenet",
                input_tensor = None,
                input_shape = (72,72,3),
                pooling = None)

model = Sequential()
model.add(xception)
model.add(Flatten())

model.add(Dense(10, activation='softmax'))

model.compile(
    loss = "categorical_crossentropy",
    optimizer = "adam",
    metrics = ['accuracy']
)

In [ ]:
model.fit(train_batch, epochs=20, validation_data=val_batch, steps_per_epoch = train_batch.n//train_batch.batch_size,
                    validation_steps = val_batch.n//val_batch.batch_size)

In [ ]:
pred = np.argmax(model.predict(testflow), axis = 1)
pred = list(pred)
classes = {value:key for (key, value) in train_batch.class_indices.items()}
predc = np.empty(300000, 'U16')
idd = 0
for i in pred:
    predc[idd] = classes[i]
    idd += 1

In [ ]:
files = []
for file in testflow.filenames:
    files.append(int(file.split('/')[1].split('.')[0]) - 1)
files = np.array(files)
    
final = np.empty(300000, dtype = 'U16')
final[files] = predc

In [ ]:
sub = pd.read_csv('../input/cifar-10/sampleSubmission.csv')
sub['label'] = final
sub.to_csv('submission.csv', index = False)